# Contingency analysis

We can define N-1 cases to be analyzed as contingencies. This means that indices of _net.line_, _net.trafo_, _net.trafo3w_ can be defined as contingencies, which are switched off one at a time. The power system is analyzed with power flow calculations, and the min/max values among all the N-1 cases are obtained for relevant variables.

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

import pandapower as pp
import pandapower.networks
import pandapower.contingency
import pandapower.control
import pandapower.timeseries
import pandapower.plotting

A dictionary that defines all the relevant N-1 cases to be analyzed is a required input parameter. Further parameters are:

* _pf_options_: dict with keyword arguments for the power flow function only for the N-0 condition
* _pf_options_nminus1_: dict with keyword arguments for the power flow function only for the N-1 situations
* _write_to_net_: whether to write the results of contingency analysis to net (in "res_" tables), default True
* _contingency_evaluation_function_: function to use for power flow calculation, default pp.runpp
* _kwargs_: additional keyword arguments passed to the power flow function for  N-0 and N-1 situations

The results of the contingency analysis are returned as a dictionary, and also written in the results tables in net:

In [2]:
net = pp.networks.case9()

nminus1_cases = {"line": {"index": net.line.index.values}}
res = pp.contingency.run_contingency(net, nminus1_cases)

numba cannot be imported and numba functions are disabled.
Probably the execution is slow.
Please install numba to gain a massive speedup.
(or if you prefer slow execution, set the flag numba=False to avoid this warning!)
numba cannot be imported and numba functions are disabled.
Probably the execution is slow.
Please install numba to gain a massive speedup.
(or if you prefer slow execution, set the flag numba=False to avoid this warning!)
numba cannot be imported and numba functions are disabled.
Probably the execution is slow.
Please install numba to gain a massive speedup.
(or if you prefer slow execution, set the flag numba=False to avoid this warning!)
numba cannot be imported and numba functions are disabled.
Probably the execution is slow.
Please install numba to gain a massive speedup.
(or if you prefer slow execution, set the flag numba=False to avoid this warning!)
numba cannot be imported and numba functions are disabled.
Probably the execution is slow.
Please install numba 

The results can be evaluated with the fuction _check\_elements\_within\_limits_. 

It uses the required input _element\_limits_, which is a dictionary with limit values for all the relevant limits. Elements with at least one limit variable set as not _nan_ are considered relevant, and elements with all limit variables set to _nan_ are ignored. 

The function returns True if all results are within specified limits, and False in case of any violations:

In [3]:
element_limits = pp.contingency.get_element_limits(net)
pp.contingency.check_elements_within_limits(element_limits, res, True)

False

The contingency analysis determined violations of limits. The function _report\_contingency\_results_ can be used to print logging statements, informing the user of the details of the limit violations:

In [4]:
pp.contingency.report_contingency_results(element_limits, res)

In [7]:
res

{'bus': {'index': array([0, 1, 2, 3, 4, 5, 6, 7, 8]),
  'max_vm_pu': array([1.        , 1.        , 1.        , 0.9995976 , 0.98259985,
         1.00545019, 0.98235029, 0.99537966, 0.962154  ]),
  'min_vm_pu': array([1.        , 1.        , 1.        , 0.95247811, 0.9075782 ,
         0.98234714, 0.93625601, 0.95610063, 0.79400736]),
  'vm_pu': array([1.        , 1.        , 1.        , 0.98700685, 0.97547218,
         1.00337544, 0.98564488, 0.99618525, 0.95762104])},
 'line': {'index': array([0, 1, 2, 3, 4, 5, 6, 7, 8]),
  'causes_overloading': array([False, False, False, False, False, False, False, False, False]),
  'cause_element': array(['line', 'line', 'line', 'line', 'line', 'line', 'line', 'line',
         'line'], dtype=object),
  'cause_index': array([6, 5, 7, 5, 5, 7, 8, 8, 6]),
  'max_loading_percent': array([95.94439381, 44.64572654, 97.85605747, 30.30775936, 75.44097543,
         66.194057  , 72.44502162, 67.82260533, 60.60208968]),
  'min_loading_percent': array([30.6411

## Time-series simulation

The function _run\_contingency_ can be passed to _run_timeseries_. 

The parameters for _run\_contingency_ can be passed as kwargs to the _run\_timeseries_ function as the parameter _run\_control\_fct_. Note: avoid passing it as the _run_ argument, because it will cause multiple contingency calculations per time step if there are any controllers in the grid model. Here we pass _nminus1\_cases_ as a keyword argument, which is passed to run_contingency during time-series simulation.

This way, the contingency analysis is executed at every timne step of the time-series simulation. The results of the contingency analysis for every time step can be added to the output of the _output\_writer_ and analyzed after the time-series calculation.

First, we configure the time-series simulation:

In [5]:
load_profiles = pd.DataFrame(net.load.p_mw.values * (np.random.random((24, len(net.load))) * 0.4 + 0.8), 
                             index=np.arange(24), columns=net.load.index.values)
dsl = pp.timeseries.DFData(load_profiles)
pp.control.ConstControl(net, element="load", variable="p_mw", element_index=net.load.index.values, 
                        profile_name=net.load.index.values, data_source=dsl)


gen_profiles = pd.DataFrame(net.gen.p_mw.values * (np.random.random((24, len(net.gen))) * 0.4 + 0.8), 
                             index=np.arange(24), columns=net.gen.index.values)
dsg = pp.timeseries.DFData(gen_profiles)
pp.control.ConstControl(net, element="gen", variable="p_mw", element_index=net.gen.index.values, 
                        profile_name=net.gen.index.values, data_source=dsg)

ow = pp.timeseries.OutputWriter(net)
ow.log_variable("res_line", "max_loading_percent")
ow.log_variable("res_line", "min_loading_percent")
ow.log_variable("res_bus", "max_vm_pu")
ow.log_variable("res_bus", "min_vm_pu")

The time-series simulation is executed:

In [6]:
pp.timeseries.run_timeseries(net, time_steps=range(24), 
                             run_control_fct=pp.contingency.run_contingency, nminus1_cases=nminus1_cases)

  0%|          | 0/24 [00:00<?, ?it/s]numba cannot be imported and numba functions are disabled.
Probably the execution is slow.
Please install numba to gain a massive speedup.
(or if you prefer slow execution, set the flag numba=False to avoid this warning!)
numba cannot be imported and numba functions are disabled.
Probably the execution is slow.
Please install numba to gain a massive speedup.
(or if you prefer slow execution, set the flag numba=False to avoid this warning!)
numba cannot be imported and numba functions are disabled.
Probably the execution is slow.
Please install numba to gain a massive speedup.
(or if you prefer slow execution, set the flag numba=False to avoid this warning!)
numba cannot be imported and numba functions are disabled.
Probably the execution is slow.
Please install numba to gain a massive speedup.
(or if you prefer slow execution, set the flag numba=False to avoid this warning!)
numba cannot be imported and numba functions are disabled.
Probably the ex

We can aggregate the result values from the output writer and show the max. and min. values for the N-0 and N-1 situations for every time step:

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, nrows=1, figsize=(10,4))

ow.output["res_line.max_loading_percent"].max(axis=1).plot(label="max. Loading, N-1 (%)", ax=ax1, linestyle="--")
ow.output["res_line.loading_percent"].max(axis=1).plot(label="max. Loading, N-0 (%)", ax=ax1)
ow.output["res_line.min_loading_percent"].min(axis=1).plot(label="min. Loading, N-1 (%)", ax=ax1, linestyle="--")
ow.output["res_line.loading_percent"].min(axis=1).plot(label="min. Loading, N-0 (%)", ax=ax1)

ax1.legend(ncol=2, loc="upper center", bbox_to_anchor=[0.5, -0.15])
ax1.set_ylabel("Line loading (%)")
ax1.set_xlabel("Time step")

ow.output["res_bus.max_vm_pu"].max(axis=1).plot(label="max. $V_m$, N-1 (p.u.)", ax=ax2, linestyle="--")
ow.output["res_bus.vm_pu"].max(axis=1).plot(label="max. $V_m$, N-0 (p.u.)", ax=ax2)
ow.output["res_bus.min_vm_pu"].min(axis=1).plot(label="min. $V_m$, N-1 (p.u.)", ax=ax2, linestyle="--")
ow.output["res_bus.vm_pu"].min(axis=1).plot(label="min. $V_m$, N-0 (p.u.)", ax=ax2)

ax2.legend(ncol=2, loc="upper center", bbox_to_anchor=[0.5, -0.15])
ax2.set_ylabel("Bus voltage magnitude (p.u.)")
ax2.set_xlabel("Time step")

plt.tight_layout()

A more advanced plotting of results using boxplot provides more insight into every time step:

In [ ]:
def plot_results_boxplots():
    fig, ((ax11, ax12), (ax21, ax22)) = plt.subplots(ncols=2, nrows=2, sharey="row", sharex="col", figsize=(10,6))

    ow.output["res_line.max_loading_percent"].T.boxplot(ax=ax11, whis=[0,100], grid=False)
    ax11.set_title("max. Loading of lines per time step, N-1 (%)")
    ax11.set_ylabel("Line loading (%)")

    ow.output["res_line.min_loading_percent"].T.boxplot(ax=ax12, whis=[0,100], grid=False)
    ax12.set_title("min. Loading of lines per time step, N-1 (%)")

    ow.output["res_bus.max_vm_pu"].T.boxplot(ax=ax21, whis=[0,100], grid=False)
    ax21.set_title("max. $V_m$ of buses per time step, N-1 (p.u.)")
    ax21.set_ylabel("Bus voltage magnitude (p.u.)")
    ax21.set_xlabel("Time step")

    ow.output["res_bus.min_vm_pu"].T.boxplot(ax=ax22, whis=[0,100], grid=False)
    ax22.set_title("min. $V_m$ of buses per time step, N-1 (p.u.)")
    ax22.set_xlabel("Time step")

    plt.tight_layout()